# Fundamentals - Variable Sharing
If we want to use multiple languages in a single notebook we'll need a way to share variables or data between them. NET Interactive provides magic commands for this, for example #!set.

Again, we need to install the Revit kernel extension and connect to a running kernel.

In [1]:
#r "nuget:RevitInteractive, 0.1.0"

Installed Packages RevitInteractive, 0.1.0

Loading extensions from `C:\Users\sejsau\.nuget\packages\revitinteractive\0.1.0\interactive-extensions\dotnet\Jowsy.DotNet.Interactive.Extensions.dll`

Revit C# Kernel Added.

In [2]:
#!connect revit --kernel-name revit24 --revit-version 2024

Kernel added: #!revit24

We will query the revit database for elements and group them by category.

In [4]:
#!revit24   
var collector = new FilteredElementCollector( doc, uidoc.ActiveView.Id);

var query = collector
        .WhereElementIsNotElementType()
        .WhereElementIsViewIndependent()
        .ToElements();

var result = query.GroupBy(x => x.Category.Name).Select(y => new {
    Id = y.Key,
    Count = y.Count()
}).ToList();

display(result);
result

index value 0 { Id = Cameras, Count = 35 } Id Cameras Count 35 1 { Id = Walls, Count = 1120 } Id Walls Count 1120 2 { Id = Stairs, Count = 27 } Id Stairs Count 27 3 { Id = Runs, Count = 43 } Id Runs Count 43 4 { Id = Supports, Count = 1151 } Id Supports Count 1151 5 { Id = Landings, Count = 17 } Id Landings Count 17 6 { Id = Railings, Count = 131 } Id Railings Count 131 7 { Id = Handrails, Count = 121 } Id Handrails Count 121 8 { Id = Multistory Stairs, Count = 3 } Id Multistory Stairs Count 3 9 { Id = Top Rails, Count = 91 } Id Top Rails Count 91 10 { Id = Floors, Count = 176 } Id Floors Count 176 11 { Id = Slab Edges, Count = 59 } Id Slab Edges Count 59 12 { Id = Doors, Count = 141 } Id Doors Count 141 13 { Id = Divisions, Count = 10 } Id Divisions Count 10 14 { Id = Vertical Circulation, Count = 2 } Id Vertical Circulation Count 2 15 { Id = Specialty Equipment, Count = 288 } Id Specialty Equipment Count 288 16 { Id = Furniture, Count = 168 } Id Furniture Count 168 17 { Id = Rectangular Straight Wall Opening, Count = 9 } Id Rectangular Straight Wall Opening Count 9 18 { Id = Roofs, Count = 20 } Id Roofs Count 20 19 { Id = Parking, Count = 20 } Id Parking Count 20 (21 more)

The result variable does only exist in memory in the revit kernel.
If we want to use it in a F#-kernel we need to use the #!set directive, as seen below:

In [5]:
#!set --value @revit24:result --name result

If you look in the notebook's variable view you will notice that a result-variables of type System.Text.Json. NET Interactive will serialize all complex object to json when exchanging between kernels that is not running in the same process.

We can now use Plotly to print a chart.

In [6]:
#r "nuget: FSharp.Data, 6.3.0"
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"

Installed Packages FSharp.Data, 6.3.0 Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.2.0

Loading extensions from `C:\Users\sejsau\.nuget\packages\plotly.net.interactive\4.2.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [7]:
open Plotly.NET
open FSharp.Data
open FSharp.Data.JsonExtensions

let data = JsonValue.Parse(result.RootElement.GetRawText());
let xData = data.AsArray() |> Seq.map (fun item -> item.GetProperty("Id").AsString())
let yData = data.AsArray() |> Seq.map (fun item -> item.GetProperty("Count").AsInteger())
let chart =
    Chart.Column(yData, xData)
    |> Chart.withTitle "Number of Instances by Category"

chart

<!-- Plotly chart will be drawn inside this DIV -->